# REINFORCE implementation

In [2]:
import gym
import datetime as dt
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow_probability as tfp
import keras

%matplotlib inline
%load_ext line_profiler

Using TensorFlow backend.


In [3]:
import importlib
import envs
from envs.Simple2dEnv import Simple2dEnv
importlib.reload(envs.Simple2dEnv)
from envs.Simple2dEnv import Simple2dEnv

In [ ]:
# Define the parameters
action_dim = 2                           # number of assets
trading_days_per_year = 1                # Number of trading days in a year
risk_free_rate = 0.01                    # Annualized risk-free rate
t_max = 100                              # Maximum number of steps in a session
learning_rate = 0.01

# User-specified utility function
ARRA = lambda x, L : 1/L * ( x ** (1/L) - 1 )
EXPRA = lambda x, L : -np.exp( -L * x )
utility = lambda x : ARRA(x,8)

entropy_weight = 0.0

gamma = np.exp(-risk_free_rate/trading_days_per_year)

In [ ]:
env = Simple2dEnv(
                    dim=action_dim, 
                    utility=utility,  
                    risk_free_rate=risk_free_rate, 
                    trading_days_per_year=trading_days_per_year
                )

observation_shape = env.observation_space.shape

state_dim = env.observation_space.shape

obs = env.reset()

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

n_hidden_0 = 8
n_hidden_1 = 8

# <define network graph using raw tf or any deep learning library>
model = Sequential( [
            Dense(n_hidden_0,  activation='relu', input_shape=state_dim ),
            #Dense(n_hidden_1,  activation='relu'),
            Dense(action_dim,  activation='relu' ),
])

optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate )

In [ ]:
N_train = 32

for j in range(250):
    X_train = tf.stack( [ env.observation_space.sample() for _ in range(N_train) ] )
    y_train = tf.random.normal( shape=(N_train,action_dim), mean=1.0, stddev=0.1 )
    history = model.fit(
        X_train, y_train,
        epochs=1, verbose=0 )

    if j % 50 == 0:
        print( [ j, np.mean(model(X_train).numpy()) ] )
        
loss, mae, mse = model.evaluate(X_train, y_train, verbose=2)        

In [8]:
_MIN_DIRICHLET_PARAM_ = 0.01
_MAX_DIRICHLET_PARAM_ = 100

# Get the parameters describing the distribution \pi(a|s)
def get_distribution_parameters( model, _states ):
    alpha = model(_states)
    return alpha


def get_distribution( model, _states ):
    
    alpha = get_distribution_parameters( model, _states=_states )
        
    # Clip the parameters to prevent exploding gradients
    clipped_alpha = tf.clip_by_value( alpha, _MIN_DIRICHLET_PARAM_, _MAX_DIRICHLET_PARAM_ )    
    
    dist = tfp.distributions.Dirichlet(clipped_alpha)
    
    return dist


def calc_entropy( model, _states ):
    
    dist = get_distribution( model, _states=_states )
    ent = dist.entropy()
    
    return ent


# Utility function to pick portfolio weights in one given state
def choose_weights_from_policy( model, _states ):     
    dist = get_distribution( model, _states=_states )
    weights = dist.sample(1)
    weights = tf.reshape( weights, shape=(tf.size(weights),) )
    
    log_prob = dist.log_prob( weights )
    return weights, log_prob


# Utility function to pick action in one given state
def choose_action_from_policy( model, _states ):     
    weights, log_prob = choose_weights_from_policy( model, _states=_states )
    action = weights[:-1]
    return action, log_prob


# Calculate the objective function for the REINFORCE algorithm
def calc_objective_function( model, _rewards, _logprobs, _gamma ):
    
    cum_rewards = get_cumulative_rewards(_rewards=_rewards, _gamma=_gamma )
    J = tf.reduce_mean( _logprobs * cum_rewards )
    return J


def calc_loss( model, _states, _rewards, _logprobs, _gamma ):
    
    entropy = calc_entropy( model, _states )
    J = calc_objective_function( model, _rewards=_rewards, 
                                    _logprobs=_logprobs, _gamma=_gamma )
    
    loss = -J - entropy_weight * entropy
    print(loss)
    return loss


def train_step( model, optimizer, _states, _logprobs, _rewards, _gamma):
    """given full session, trains agent with policy gradient"""

    loss_fun = lambda : calc_loss( model, _states=_states, 
                        _rewards=_rewards, _logprobs=_logprobs, _gamma=_gamma )
    
    current_loss = optimizer.minimize( loss_fun, var_list=model.weights )
    return current_loss


def get_cumulative_rewards(_rewards, _gamma ):

    # Calculate the discount factor. Assumes the first rewards are earliest    
    cum_rewards = [np.nan] * len(_rewards)
    cum_rewards[-1] = _rewards[-1]
    for t in range(1, len(_rewards)):
        cum_rewards[-(t+1)] = _rewards[-(t+1)] + _gamma * cum_rewards[-t] 

    return np.array( cum_rewards )

In [9]:
def generate_session( model, t_max ):
    """play env with REINFORCE agent and train at the session end"""

    # arrays to record session
    states, actions, rewards, log_probs = [], [], [], []

    obs = env.reset()
    obs = obs[:,np.newaxis].T

    for t in range(t_max):

        # action probabilities array aka \pi(a|s)
        action, log_prob = choose_action_from_policy( model, obs )

        # Move to next state
        new_obs, r, done, info = env.step(action )
        new_obs = new_obs[:,np.newaxis].T

        # record session history to train later
        states.append(obs)
        actions.append(action)
        log_probs.append(log_prob)
        rewards.append(r)

        obs = new_obs
        if done:
            break

    state_mtx = tf.stack( states )
    action_mtx = tf.stack( actions )
    logprob_mtx = tf.stack( log_probs )
    reward_mtx = tf.stack( rewards )
    
    train_step( model, optimizer=optimizer, _states=state_mtx, 
                   _logprobs=logprob_mtx, _rewards=reward_mtx, _gamma=gamma )

    # technical: return session rewards to print them later
    return sum(rewards)

In [ ]:
from IPython.display import clear_output

mean_rewards = []
fig = plt.figure()
for i in range(100): 

    rewards = [ generate_session(model, t_max) for _ in range(20)]  # generate new sessions
    mean_rewards.append( np.mean(rewards) )
    clear_output()
    plt.plot( mean_rewards )
    plt.pause(0.05)
    
    print( "{}: mean reward: {}".format(i, np.mean(rewards)))

In [ ]:
# arrays to record session
states, actions, rewards, log_probs = [], [], [], []

obs = env.reset()
obs = obs[:,np.newaxis].T

for t in range(t_max):

    # action probabilities array aka \pi(a|s)
    action, log_prob = choose_action_from_policy( model, obs )

    # Move to next state
    new_obs, r, done, info = env.step(action )
    new_obs = new_obs[:,np.newaxis].T

    # record session history to train later
    states.append(obs)
    actions.append(action)
    log_probs.append(log_prob)
    rewards.append(r)

    obs = new_obs
    if done:
        break

state_mtx = tf.stack( states )
action_mtx = tf.stack( actions )
logprob_mtx = tf.stack( log_probs )
reward_mtx = tf.stack( rewards )

loss = train_step( model, optimizer=optimizer, _states=state_mtx, 
                   _logprobs=logprob_mtx, _rewards=reward_mtx, _gamma=gamma )

In [ ]:
entropy = calc_entropy( model, state_mtx )
print(np.mean(entropy) / np.mean(reward_mtx) )

In [ ]:
_states = state_mtx
_rewards = reward_mtx 
_logprobs = logprob_mtx
_gamma = gamma
entropy = calc_entropy( model, _states )
J = calc_objective_function( model, _rewards=_rewards, 
                                _logprobs=_logprobs, _gamma=_gamma )

-J - entropy_weight * entropy
[ np.mean(-J), np.mean(- entropy_weight * entropy) ]

In [ ]:
plt.scatter( action_mtx.numpy(), -J )
plt.ylim( -0.002, 0.002 )

In [ ]:
entropy = calc_entropy( model, state_mtx )
J = calc_objective_function( model, action_mtx, reward_mtx, gamma )
loss = -J - entropy_weight * entropy

In [ ]:
tfp.distributions.Dirichlet( [ .01, 100 ] ).entropy()

In [ ]:
obs = env.reset()
obs = obs[:,np.newaxis].T

mu = env.state_params['mu']
sigma = env.state_params['sigma']
vols = np.sqrt(np.diag(sigma))
print(mu)
print(vols)
print(mu/vols)

In [ ]:
new_obs, r, done, info = env.step([1.])
print( [ new_obs[0], r ] )

In [ ]:
plt.scatter( action_mtx.numpy(), reward_mtx.numpy())

In [ ]:
tfp.distributions.Normal?

In [ ]:
cum_rwds = get_cumulative_rewards(_rewards=rewards, _gamma=gamma )
plt.hist( logprob_mtx.numpy().flatten() )
#plt.hist( cum_rwds.flatten() * logprob_mtx.numpy().flatten() )

In [ ]:
cum_rwds